# Setup

In [1]:
# data processing
import pandas as pd

# regular expressions
import re

# operating system
import os

# web scraping
import urllib
# fetch the URL
import urllib.request
from bs4 import BeautifulSoup

# accessing APIs and URLs
import requests

#import the class fetch from Object Oriented Programming
import data_collection_oop

#read/write Excel files
import openpyxl

# Data collection

# Extract links from web

In [2]:
#define URL for scraping
theurl = "https://www.americanrhetoric.com/barackobamaspeeches.htm"
thepage = urllib.request.urlopen(theurl)

#Cooking the Soup
soup = BeautifulSoup(thepage,"html.parser")

web_list=[]
#Scraping "Link" (href)
for a_href in soup.find_all("a", href=True):
    #print(a_href["href"])
    web_list.append(a_href["href"])

In [3]:
# extract all links from website
speech=[]
for i in web_list:
    if i.startswith("speeches"):
        speech.append(i)
        
# only retain links that contain speech content    
speech_web=[]
for i in speech:
    if i.endswith('htm'):
        speech_web.append(i)

In [4]:
#only retain links starting from 2008
start = speech_web.index('speeches/barackobama/barackobamainauguraladdress.htm')
speech_president = speech_web[start:]

# remove replicate
new = []
for i in speech_president:
    if i not in new:
        new.append(i)

# concatenate basic url and path
speech_url = []
for val in range(len(new)):
    speech_url.append("https://www.americanrhetoric.com/" + str(new[val])) 
    
# print number of links    
len(speech_url)

428

# Fetch data from web

In [5]:
# extract the speech content from the links
speeches = [data_collection_oop.fetch.extract_text(url) for url in speech_url]

#print number of speeches 
len(speeches)

428

In [6]:
# extracted the title and content of each speech
speech_content = []
speech_title = []

# save them into separate lists
for i in range(len(speeches)):
    speech_content.append(speeches[i][1])
    speech_title.append(speeches[i][0])
    
speech_title = [data_collection_oop.fetch.title(x) for x in speech_title]

In [7]:
## delete something like cd, pdf ... in the speech content
speech_content = [data_collection_oop.fetch.delet(speech) for speech in speech_content]

# List Date ( 2009.01.20 – 2017.01.20 )

In [9]:
year = ["2009","2010","2011","2012","2013","2014","2015","2016","2017"]

# find the date corresponding to Obama's speeches using BeautifulSoup
date_list = []
for i in range(len(year)):
    date_list.extend(soup.find_all(string=re.compile(year[i])))

In [11]:
# delete non-relevant data from the date list
no = ['Press Conference on 2010 Budget Sent to Congress','Announcement of 2012 Presidential Candidacy',
     'Honoring Golden State Warriors 2015 NBA Champs','Press Conference Following 2016 NATO Summit']
for i in range(len(no)):
    date_list.remove(no[i])
len(date_list)

435

In [14]:
# Convert the number into an string
date = date_list[date_list.index("20 Jan 2009"):]
date = [str(x) for x in date]

#  Create a Dataframe

In [16]:
df_speech = pd.DataFrame({'date': date,'title':speech_title,'content':speech_content}
                         )
df_speech.head()

,date,title,content
0,20 Jan 2009,First Presidential Inaugural Addre,[Chief Justice John G. \r\n\t\tRoberts adminis...
1,24 Jan 2009,>American Rhetoric: Barack Obama: First Presid...,: First Presidential Weekly Address \r\n(01-24...
2,26 Jan 2009,Al-Arabiya Television Intervi,- Al-Arabiya Television InterviewBarackObamaAl...
3,04 Feb 2009,Remarks on Executive Compensati,- Remarks on Executive CompensationBarackObama...
4,09 Feb 2009,First Presidential Prime Time Press \r\nConferen,"Good \r\n\t\tevening, everybody. Please be sea..."


In [19]:
# write data frame to excel file
os.makedirs('output', exist_ok = True)
df_speech.to_excel("output/obama_speeches_dataframe.xlsx")